#### This notebook is used to perofrm the supervised tweet classification experment for the Birdwatch metric. 

In [9]:
import pandas as pd
import numpy as np
import sklearn as sk
from collections import Counter
from sklearn.metrics import classification_report
import pickle
import math
from tqdm import tqdm

The file 'hawkeye_metric_tweet_vectors_supervised.pickle' stores a vector for each tweet. This file is generated by the 'tweet_vectors_baseline.py' script. 

In [5]:
with open('results/hawkeye_metric_tweet_vectors_supervised.pickle', 'rb') as handle:
    tweet_vectors = pickle.load(handle)

notes = pd.read_csv("..//data//notes-00000-13-04-21.tsv", sep='\t')
ratings = pd.read_csv("..//data//ratings-00000-13-04-21.tsv", sep='\t')

In [7]:
tweet_annotation_df_next = pd.read_csv('tweet_groudtruth_annonation_next.csv',encoding='utf-8-sig')
tweet_annotation_df = pd.read_csv('tweet_groudtruth_annonation.csv',encoding='utf-8-sig')
tweet_annotation = pd.concat([tweet_annotation_df, tweet_annotation_df_next])

The true labels for tweets are obtained form the above annotation sheets. The  predicted  labels  for  the  tweets  for  both  the are determined as follows. For  the  Birdwatch  system,  we  count  the  number  of  helpful notes as the notes that have a helpfulness ratio of at least 0.84 (this  threshold  is  used  by  Birdwatch).  For  each  tweet,  if  the number of  helpful notes  that labeled  the tweet  as misleading are   more   than   or   equal   to   the   number   of   helpful   notes that  labeled  the  tweet  as  not-misleading,  we  say  Birdwatch classifies  the  tweet  as  misleading.  We  use  this  technique  as Birdwatch does not define a standard way for tweet labeling

In [10]:
y_true,y_pred = [],[]
for idx,row in tweet_annotation.iterrows():
    
    y_true.append(row['human_annotation'])
    notesForTweet = notes.loc[notes['tweetId'] == int(row['tweet_id'][:-2])]
    
    ratingsWithNotesForTweet = notesForTweet.set_index('noteId').join(ratings.set_index('noteId'), lsuffix="_note", rsuffix="_rating", how='left')
    ratingsWithNotesForTweet['numRatings'] = ratingsWithNotesForTweet.apply(lambda x: 0 if math.isnan(x['helpful']) else 1, axis=1)    
    
    scoredNotes = ratingsWithNotesForTweet.groupby(['noteId']).agg({'helpful':'sum', 'numRatings' : 'sum', 'classification':'first'})
    scoredNotes['helpfulnessRatio'] = scoredNotes['helpful']/scoredNotes['numRatings']
    scoredNotes = scoredNotes[scoredNotes['helpfulnessRatio'] >= 0.84]

    scoredNotes_misleading = scoredNotes.loc[scoredNotes['classification'] == 'MISINFORMED_OR_POTENTIALLY_MISLEADING']
    scoredNotes_notmisleading = scoredNotes.loc[scoredNotes['classification'] == 'NOT_MISLEADING']
    
    if len(scoredNotes_misleading) >= len(scoredNotes_notmisleading): # >= scoredNotesNotMisleading['helpful'].sum():
        y_pred.append(0)
    else:
        y_pred.append(1)

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.99      0.40       124
           1       0.75      0.01      0.02       376

    accuracy                           0.25       500
   macro avg       0.50      0.50      0.21       500
weighted avg       0.63      0.25      0.11       500

